In [1379]:
"""
Скрипт для обработки общйе таблицы Омского ЦОПП. 
"""

'\nСкрипт для обработки общйе таблицы Омского ЦОПП. \n'

In [1380]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import tkinter
import sys
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time

pd.options.mode.chained_assignment = None  # default='warn'
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import copy
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import random

In [1381]:
def check_data(cell):
    """
    Метод для проверки значения ячейки
    :param cell: значение ячейки
    :return: число в формате int
    """
    if cell is np.nan:
        return 0
    if cell.isdigit():
        return int(cell)
    else:
        return 0

In [1382]:
def group_text_value(value):
    """
    функция для группировки текстовых данных, группирует только заполненные строки
    """
    tmp_set = set(value.tolist()) # создаем множество
    tmp_set.discard('Не заполнено')
    tmp_set.discard('0')
    return ';'.join(tmp_set)



In [1383]:
def check_sameness_range(df:pd.DataFrame,names_columns:list,border,amendment):
    
    """    
    Функция для проверки единнобразия значений в колонке по диапазонам
    """
    # получаем поправки для того чтобы диапазон ошибки указывался корректно
    first_correct = amendment[0]
    second_correct = amendment[1] 
    _error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки']) # датафрейм для ошибок
    for column in names_columns:
        _temp_set = set(df[column].tolist())
        if len(_temp_set) != 1:
            temp_error_df = pd.DataFrame(data=[[f'Диапазон строк {border + first_correct} - {border+second_correct},Колонка {column}',
                                        'В указанном диапазоне и колонке  должны быть одинаковые значения во избежание ошибок при подсчете,если указан диапазон вне таблицы то удалите строки этого диапазона']],
                                 columns=['Строка или колонка с ошибкой',
                                          'Описание ошибки'])
            _error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
    return _error_df
        

In [1384]:
def check_horizont_all_sum_error_omsk(df: pd.DataFrame, tup_exluded_cols: tuple, name_itog_cols):
    """
    Функция для проверки горизонтальных сумм по всей строке
    сумма в колонке 05 должна быть равна сумме всех колонок за исключением 07 и 15 Пример
    """
    # получаем список колонок
    all_sum_cols = list(df)
    # удаляем колонки
    for name_cols in tup_exluded_cols:
        all_sum_cols.remove(name_cols)
    # удаляем итоговую колонку
    all_sum_cols.remove(name_itog_cols)

    # получаем сумму колонок за вычетом исключаемых и итоговой колонки
    df['Сумма'] = df[all_sum_cols].sum(axis=1)
    # Проводим проверку
    df['Результат'] = df[name_itog_cols] == df['Сумма']
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
    # получаем датафрейм с ошибками и извлекаем индекс
    df = df[df['Результат'] == 'Неправильно'].reset_index()
    df.to_excel('dsds.xlsx',index=False)
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки', ])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist()  # делаем список
    finish_lst_index = list(map(lambda x: x + 1, raw_lst_index))
    finish_lst_index = list(map(lambda x: f'Строка {x+6}', finish_lst_index))
    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df[
        'Описание ошибки'] = f'Не выполняется условие: гр. {name_itog_cols} = сумма остальных гр. за искл.{tup_exluded_cols}!!!'
    return temp_error_df

In [1385]:
def check_error_omsk(df:pd.DataFrame,border,size_range,amendment):
    """
    Функция для проверки на ошибки
    """

    error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки']) # датафрейм для ошибок
    quantity_range = df.shape[0] // size_range #  получаем количество проходов которые нужно пройти
    sameness_columns = ['1','2','3'] # список колонок которые нужно проверить на единообразие внутри диапазона
    
    # данные для проверки горизонтальной суммы по строкам колонка 6 должна быть равна сумме всех колонок за исключением excluded_cols
    itog_cols = '6' # номер колонки по которой будет проверятся сумма по горизонтали 
    excluded_cols = ('1','2','3','4','5','8','9','10','11','34') # колонки которые не будут учитываться
    
    for i in range(quantity_range):
        temp_df = df.iloc[border:border+size_range,:]
        error_sameness_df = check_sameness_range(temp_df.copy(),sameness_columns,border,amendment) # проверка на единообразие
        error_df = pd.concat([error_df, error_sameness_df], axis=0, ignore_index=True)
     
        border += 5 # смещаем диапазон
    
    
        
    all_horizont_error_df = check_horizont_all_sum_error_omsk(df.copy(),excluded_cols,itog_cols)
    error_df = pd.concat([error_df, all_horizont_error_df], axis=0, ignore_index=True)
    
    
    
    
    
    
    return error_df
        
        
        
        
    
    
    
    

In [1386]:
path_data_omsk = 'data/Омск/Трудоустройство_1_07_2023_выпуск_2022.xlsx'
path_to_end_folder = 'data'
# создаем датафрейм для регистрации ошибок
error_df = pd.DataFrame(columns=['Строка или колонка с ошибкой', 'Описание ошибки'])

In [1387]:
# загружаем файл
df = pd.read_excel(f'{path_data_omsk}',dtype=str,skiprows=5)

In [1388]:
df=df.iloc[:,:34] # убираем строки проверки

In [1389]:

# заменяем названия колонок
df.columns = list(map(str,range(1,df.shape[1]+1)))

# колонки к которым надо применить числовые суммирование
columns_to_apply = ['6','7','8','10','11','12','13','14','15','16','17','18','19','20','21','22','23'
                    ,'24','25','26','27','28','29','30','31','32','33']
df[columns_to_apply] = df[columns_to_apply].applymap(check_data) # обрабатываем


In [1390]:
df.shape

(2030, 34)

In [1391]:
# Проверяем на количество строк, должно быть кратно 5
if df.shape[0] % 5 !=0:
    temp_error_df = pd.DataFrame(data=[['',
                                        'Количество строк не кратно 5 !!! Возможно пропущены строки с данными или есть лишние строки в конце таблицы']],
                                 columns=['Строка или колонка с ошибкой',
                                          'Описание ошибки'])
    error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)

In [1392]:
# Проверяем ошибки
tup_correct = (7, 11)  # создаем кортеж  с поправками
border = 0 # счетчик строк
size_range = 5 # сколько строк занимает каждый диапазон
check_error_df= check_error_omsk(df.copy(),border,size_range,tup_correct)
error_df = pd.concat([error_df, check_error_df], axis=0, ignore_index=True)


In [1393]:
df.head(1)

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,"БПОУ ""Седельниковский агропромышленный техникум""",23.01.03,Автомеханик,01,Всего,22,5,1,"1 чел -ПАО ""ОПК-Сатурн"" ОМКБ",4,...,0,0,0,1,0,0,0,0,0,Опрос-6 чел


In [1394]:
# заполняем нан в текстовых колонках
df['9'] = df['9'].fillna('Не заполнено')
df['34'] = df['34'].fillna('Не заполнено')

In [1395]:
# проводим группировку
df = df.groupby(['2','3','4','5']).agg({'6':sum,'7':sum,'8':sum,'9':sum,'10':sum,'11':sum,'12':sum,'13':sum,'14':sum,'15':sum,'16':sum,
                                   '17':sum,'18':sum,'19':sum,'20':sum,'21':sum,'22':sum,'23':sum,'24':sum,'25':sum,'26':sum,'27':sum,
                                   '28':sum,'29':sum,'30':sum,'31':sum,'32':sum,'33':sum,'9':group_text_value,'34':group_text_value})

In [1396]:
# вытаскиваем индексы
df = df.reset_index()

In [1397]:
error_df

,Строка или колонка с ошибкой,Описание ошибки
